In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import datetime as dt
from scipy import stats
from sklearn import preprocessing
import matplotlib as mpl
from scipy.stats import skew
import geopandas
import seaborn as sns

## Plotting parameters
mpl.rcParams['pdf.fonttype'] = 42

## Cubic feet to cubic meters conversion factor
cfs_2_cms = 0.0283168466

In [54]:
def readModelFile(i):
    try:
        df = pd.read_csv('../data/pnwNP_modeledData_NWM3/'+str(pnwNP['gage'][i])+".csv")
        df['time'] = pd.to_datetime(df['time'])
       
    ### Deliniate time periods
        df['climate_year'] = df.time.dt.year.where(df.time.dt.month < 4, df.time.dt.year + 1)
        df.loc[(df.time.dt.month > 7) & (df.time.dt.month < 10), "flow_season"] = "late_summer"
        df.loc[(df.time.dt.month > 2) & (df.time.dt.month < 6), "season"] = "spring"
        df.loc[(df.time.dt.month > 5) & (df.time.dt.month < 9), "season"] = "summer"
        df.loc[(df.time.dt.month > 8) & (df.time.dt.month < 12), "season"] = "fall"
        df.loc[(df.time.dt.month > 11), "season"]  = "winter"
        df.loc[(df.time.dt.month < 3), "season"] = "winter"
        df['month'] = df.time.dt.month
        df["DOY"] = df.time.dt.day_of_year
        
        # df = df.drop(['Unnamed: 0.1'],axis=1)
        df = df[df["streamflow_NWIS"].notna()].reset_index(drop=True)
        return df

    except:
        pass

In [55]:
def calcLowFlowTiming(df):
    import warnings
    warnings.filterwarnings("ignore")
    '''
    Read in data files for gages and calculate low flow timing
    '''
    
    out7Q = pd.DataFrame()
    outLowFLow = pd.DataFrame()

    for column in df[["streamflow_VIC","streamflow_PRMS",'streamflow_NWM2d1','streamflow_NWM3','streamflow_NWIS']]:
        
        ## Find date of 7-day low flow
        stat = "7Q_"
        df['7Q'] = df[column].rolling(window=7).mean()
        tt = df[['climate_year','DOY','time','7Q']]
        tt = tt.loc[tt.groupby(['climate_year'])["7Q"].idxmin()].set_index('climate_year').dropna()
        tt = tt[df['7Q'].isnull().groupby(df['climate_year']).sum() <= 36.6]        
        # tt = tt[["DOY","time"]].reset_index(drop=True).rename(columns={'DOY': column + "_" + stat + "DOY", 'time': column + "_" + stat + "time"})
        tt = tt[["DOY"]].reset_index(drop=True).rename(columns={'DOY': column + "_" + stat + "DOY"})
        
        out7Q  = pd.concat([out7Q,tt],axis=1)
        
        ## Find first date of annual low-flow
        stat = "LowFlow_Start_"

        df.loc[df[column]<=(df[column].quantile(0.25)),"new"] = 1
        tt = df[['climate_year','DOY','time','new']].dropna()
        tt = tt.loc[tt.groupby(['climate_year'])["new"].idxmin()].set_index('climate_year').dropna()
        tt = tt[df[column].isnull().groupby(df['climate_year']).sum() <= 36.6]
        # tt = tt[["DOY","time"]].reset_index(drop=True).rename(columns={'DOY': column + "_" + stat + "DOY", 'time': column + "_" + stat + "time"})
        tt = tt[["DOY"]].reset_index(drop=True).rename(columns={'DOY': column + "_" + stat + "DOY"})
        
        outLowFLow  = pd.concat([outLowFLow,tt],axis=1)
        

    outLowFLow["gage"] = int(df["gage"][0])
    out7Q["gage"] = int(df["gage"][0])
    
    out7Q = out7Q.melt(id_vars=["gage"]).dropna()
    outLowFLow = outLowFLow.melt(id_vars=["gage"]).dropna()
    return out7Q, outLowFLow


In [57]:
from tqdm import tqdm

pnwNP = pd.read_csv("../data/KGE_gages.csv")

out7Q_comp = pd.DataFrame([])
outLowFLow_comp = pd.DataFrame([])

for i in tqdm(range(len(pnwNP))):
    df = readModelFile(i)
    out7Q, outLowFLow  = calcLowFlowTiming(df)
    output = pd.concat([output, out], ignore_index=True)
    out7Q_comp = pd.concat([out7Q_comp, out7Q], ignore_index=True)
    outLowFLow_comp = pd.concat([outLowFLow_comp, outLowFLow], ignore_index=True)
    del(out7Q,outLowFLow,df)
    


100%|████████████████████████████████████████████████████████████████████████████████| 166/166 [00:20<00:00,  8.06it/s]


In [58]:
out7Q_comp.to_csv("../data/LowFlow7QTiming_stats.csv",index_label=None)
outLowFLow_comp.to_csv("../data/LowFlowTiming_stats.csv")